In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib

# Python inclass practice 6: Basic statistics

## 1. First, let's revisit how to download USGS data!
### 1.1. Define site-specific info

In [ ]:
args = {
    'site_no': '04216000',
    'begin_date': '1930-09-01',
    'end_date': '2023-08-31'
}

In [ ]:
query = urllib.parse.urlencode(args)

In [ ]:
query

## 1.2. Create the url and access the data using urllib

In [ ]:
verde_url = (
    f'https://waterdata.usgs.gov/nwis/dv?'
    f'cb_00060=on&format=rdb&referred_module=sw&{query}'
)
print(verde_url)

## 1.3. Read the data

In [ ]:
response = urllib.request.urlopen(verde_url)

# Anyways, let's walk through a few of them:
#  - comment='#': Lines beginning with a '#' are comments that pandas should ignore
#  - sep='\s+': The data representing columns are separated by white space
#  - names: The names of the columns. I set these because the USGS ones are trash
#  - index_col=2: Set the 3rd column as the index (that is, "date")
#  - parse_dates=True: Try to make dates the correct data type, didn't work here but a good idea
#  - date_format='yyyy-mm-dd': Display the format of date
#  - engine='python': Python engine is currently more feature-complete

df = pd.read_table(
    response,
    comment='#',
    sep='\s+',
    names=['agency', 'site', 'date', 'streamflow', 'quality_flag'],
    index_col=2,
    parse_dates=True,
    date_format='yyyy-mm-dd',
    engine='python'
)

In [ ]:
# discard the first two rows
df = df.iloc[2:]

In [ ]:
# Now convert the streamflow data to floats and
# the index to datetimes. When processing raw data
# it's common to have to do some extra postprocessing
df['streamflow'] = df['streamflow'].astype(np.float64)
df.index = pd.DatetimeIndex(df.index)
df.head()

### 1.4. Practice #1: Please download following dataset
Site id: 04215000 (Cayuga Creek neear Lancaster NY) </br>
Start date: 1930-09-01 </br>
End date: 2023-08-31

In [1]:
# INSERT your code here
# save data for Cayuga sites to **df1**

## 1.5. Concatenate multi-site information

First, we only want streamflow data,so when concatenating multi-site information, we only only keep the column "streamflow". However, if both sites use "streamflow" as their column names, how can we tell which is which? We also need to rename the column names to their USGS site ID.

In [ ]:
df_n = df[['streamflow']].rename({'streamflow':'04216000'},axis=1)
df_n

#### 1.5.0. Practice #2: Like we did to `df`, extract "streamflow" column from `df1`, and rename the column "streamflow" to its corresponding USGS site ID "04215000"

##### _n represents Niagara River and _c represents Cayuga Creek

In [ ]:
df_c = #INSERT your code here

### 1.5.1. concatenate two dataframes `df_n` and `df_c`

In [ ]:
df_concat = pd.concat([df_n,df_c],axis=1)

### 1.5.2. We can quickly take a look at the two time series by simply using `df.plot`!

In [ ]:
df_concat.plot()

### 1.5.3. Practice #3: We noticed that there is some missing data from the late 1960s to early 1970s. For simplicity, we will focus on the period when both sites have data. Let's select the data between `1980-01-01` and `2022-12-31`.
If you don't remember how to select time series, check Section **2.4.2** in https://github.com/act-hydro/GLY606_2024/blob/main/in_class_practice/python_practice/python_inclass_5_dataframe.ipynb

In [ ]:
df_concat_sel = #Insert your code here

### 1.5.4. Last sanity check: we do not want have any missing data in the dataset
`df.dropna()` will automatically drop the rows with missing values. </br>
If we want to drop the columns with missing values, we should use `df.dropna(axix=1)`

In [ ]:
df_concat_sel = df_concat_sel.dropna()

# 2. Statistics calcluations!

## 2.1. Mean values

In [4]:
df_concat_sel = pd.read_csv("python_inclass_practice_6.data.csv",index_col=0,parse_dates=True)

In [5]:
df_concat_sel.iloc[]

,04216000,04215000
date,,
1980-01-01,227000.0,138.0
1980-01-02,230000.0,123.0
1980-01-03,223000.0,102.0
1980-01-04,214000.0,51.0
1980-01-05,214000.0,45.0
...,...,...
2022-12-27,213000.0,142.0
2022-12-28,209000.0,182.0
2022-12-29,206000.0,278.0


In [ ]:
df_concat_mean = df_concat_sel.mean()
df_concat_mean

### 2.1.1. How do we quickly visualize the mean values in a time series plot?

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[5,3],dpi=300)
df_concat_sel[['04216000']].plot(ax=ax,lw=0.5)
ax.set_title('USGS streamflow @ Niagara River (Buffalo, NY)\n Site No 04216000')
ax.set_ylabel('streamflow [cfs]')
ax.axhline(df_concat_mean['04216000'],c='orange')

### 2.1.2. Practice #4: A quick recap of subplots!
Since Niagara River and Cayuga Creek near Lancaster NY have different flow volumns, it is preferrable to plot them separately. So let's use `plt.subplots` to show the time series and mean values for both sites. </br>

If you forgot how subplots works, check the bottom part of inclass practice 3 https://github.com/act-hydro/GLY606_2024/blob/main/in_class_practice/python_practice/python_inclass_3_intro_to_matplotlib.ipynb

In [ ]:
# Please insert your code below 

### 2.1.3. What we can learn from the time series plots above?
Drainage area for Niagara River in Buffalo NY: 263,700 square miles </br>
Drainage area for Cayuga Creek near Landcaster NY: 96.4 square miles </br>

Streamflow in smaller river basins like Cayuga Creek is usually more responsive to precipitation events, leading to a more spiky high flow events.


## 2.2. standard deviation (SD)
To calculate the standard deviation for a column in dataframe, we can simply use `df.std()`

In [ ]:
df_concat_sd = df_concat_sel.std(axis=0)

In [ ]:
df_concat_sd

## 2.3. Coefficient of variation
CV = (Standard Deviation) / (Mean) </br> 
A higher CV indicates greater relative variability in the data, meaning the data points are spread further apart compared to the mean

In [ ]:
df_concat_coeff_variation = df_concat_sd / df_concat_mean

In [ ]:
df_concat_coeff_variation

### Once again, Cayuga Creek has a much higher coefficient of variation than Niagara River, which echos our observations about the streamflow timeseries.

## 2.4. Box plots

We can use box plots to show the spread of the datasets!

In [ ]:
fig,axes = plt.subplots(1,2,figsize=[5,6],dpi=100)
axes[0].boxplot(df_concat_sel[['04216000']])
axes[0].set_ylabel("streamflow [cfs]")
axes[0].set_xticks([1],['0421600'])

axes[1].boxplot(df_concat_sel[['04215000']])
axes[1].set_xticks([1],['0421500'])
axes[1].set_ylim(0,)
# axes[1].set_yscale('log')
plt.tight_layout()


### 2.4.1. Practice #5: log-scale of y-axis
The box plot in the Cayuga River is largely influenced by the high flow events. What if we change the y-axis to log-scale? Please uncomment `ax.set_yscale('log')` and see what changes this line makes.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[6,10],dpi=300)
ax.boxplot(df_concat_sel[['04215000']],flierprops={'marker':'+','linewidth':0.2})
ax.set_xticks([1],['0421500'])
# ax.set_yscale('log')


# 3. PDF & CDF

## 3.1. Probability Density Function

For discrete sample data, we usually use histogram `plt.hist` to visualize it. </br>
Note: this plotting function has an option called `density`. If `density=False`, the plot will show the count falling within each bin. If `density=True`, the plot will show the probability density.

In [ ]:
fig,ax = plt.subplots(1, 1,figsize=[5,4],dpi=200)
ax.hist(df_concat_sel['04216000'],bins=np.arange(100000,350000,10000),density=True)
ax.set_title("PDF for streamflow in Niagara River")

### 3.1.1. It is important to choose bin size and scales wisely

For example, the streamflow regime in Cayuga River is very spiky. Most of the time, the flow is between 0-100 cfs, but it also had some very high flow days with thousands cfs. Therefore, if we use normal scale plots with fixed bins, it will look like left plot below. However, if we use log-scale for x-axis with smart selection of bin width, the plot looks much better (like the one on the right).

In [ ]:
fig,axes = plt.subplots(1, 2,figsize=[10,4],dpi=200)

axes[0].hist(df_concat_sel['04215000'],bins=np.arange(100,4000,100),
             density=True)
axes[1].hist(df_concat_sel['04215000'],bins=np.concatenate([[0,1,2,5,10,20,50,80],
                                                            np.arange(100,4000,100)]),
             density=True)
axes[1].set_xscale('log')
axes[0].set_title("Linear-scale")
axes[1].set_title("Log-scale")

fig.suptitle("PDF for streamflow in Cayuga Creek")

## 3.2. Cumulative Distribution Function

If we have a discrete array of samples, and we would like to know the CDF of the sample, then we can just sort the array. If we look at the sorted result, we'll realize that the smallest value represents 0% , and largest value represents 100 %. Then all other arrays are assigned with evenly distributed probabilities between 0% and 100%.

In [ ]:
# Calculate the cumulative proportion of the data that falls below each value
cumulative = np.linspace(0, 1, len(df_concat_sel['04216000']))

# Sort the data in ascending order
sorted_data = np.sort(df_concat_sel['04216000'])

# Calculate the cumulative proportion of the sorted data
cumulative_data = np.cumsum(sorted_data) / np.sum(sorted_data)

In [ ]:
# Plot the CDF
plt.figure(dpi=300)
plt.plot(sorted_data, cumulative_data, label='streamflow')
plt.ylim(0,1)

# Add labels and title
plt.xlabel("Streamflow (cfs)")
plt.ylabel("Cumulative Proportion")
plt.title("Cumulative Distribution Function (CDF) of Mean daily flow")
plt.legend()
plt.show()

## 3.2.1. Practice #6: plot CDF for Cayuga River. 

In [ ]:
# Please insert your code here

## 3.3. 100-year return floods

As we discussed in class, to calculate the 100-year return floods, we can follow the three steps:
1. Identify annual peak flow `groupby`
2. Calculate the CDF for annual peak flow
3. Calculate the Annual exceedance probability (AEP)
4. In the AEP, find the flow value corresponds to AEP = 1%.

#### 3.3.1. Step 1: Identify annual peak flow

In [ ]:
# Step 1: Identify annual peak flow
df_annual_max = df_concat_sel.groupby(df_concat_sel.index.year).max()
df_annual_max

#### 3.3.2. calculate cdf

In [ ]:
# 2: calculate cdf
# Calculate the cumulative proportion of the data that falls below each value
cumulative_n = np.linspace(0, 1, len(df_annual_max['04216000']))

# Sort the data in ascending order
sorted_data_n = np.sort(df_annual_max['04216000'])

# Calculate the cumulative proportion of the sorted data
cumulative_data_n = np.cumsum(sorted_data_n) / np.sum(sorted_data_n)

In [ ]:
cumulative_data_n

#### 3.3.3. calculate AEP

In [ ]:
# 3: calculate AEP
aep_n = 1 - cumulative_data_n

In [ ]:
# visualize AEP
plt.figure(dpi=200)
plt.plot(aep_n, sorted_data_n,  label='Niagara River Peak Flow')
plt.scatter(aep_n, sorted_data_n)
plt.axvline(0.01,lw=0.5,c='orange')
plt.xlim(0,1)
aep_list = np.array([0.01,0.05,0.1,0.2,0.5,1])
aep_str = []
for i in aep_list:
    if i*100<1:
        aep_str.append("%s%%"%(i*100))
    else:
        aep_str.append("%i%%"%(i*100))
plt.xticks(aep_list,aep_str)
plt.grid(lw=0.4)

# Add labels and title
plt.ylabel("Streamflow (cfs)")
plt.xlabel("Annual Exceedance Probability")
plt.title("Annual Exceedance Probability")
plt.legend()
plt.show()

#### 3.3.4. find the flow corresponds to AEP = 0.01

In [ ]:
# 4: find the flow corresponds to AEP = 0.01

# Basically, we need to find the y value that 
# corresponds to the intersection of orange line 
# and blue dotted line above

# asp_n[::-1] reverse this array, for example
# a = [0,1,2], and a[::-1] will output [2,1,0]

# The reason behind reverse the asp_n is
# np.interp requires an monotonically increasing 
# sample points but our asp_n was in monotonically
# decreasing order.

flow_100yr_flood_n = np.interp(0.01, aep_n[::-1], sorted_data_n[::-1])
print("The flood with 100-year recurrence interval is %.2f cfs"%(flow_100yr_flood_n))

## 3.4.1. Practice #7: Please output the 100-year flood for Cayuga River

In [ ]:
# Insert your code here

## 3.5. 7Q10 for evaluating low flows!
### 3.5.1. We first need to calculate the minimum annual 7-day average flow
The syntax for rolling average is quite simple! </br>
We can use `df.rolling()` to calculate the rolling average. </br>

In [ ]:
df_rolling_7day_mean = df_concat_sel.rolling(7,center=True).mean()

In [ ]:
df_rolling_7day_mean

# Note: the first three days and last three days does not have data!
Because data was not available for the nearby 7-days at those dates.

### 3.5.2. Here we simply drop the NAN in the moving average

In [ ]:
df_rolling_7day_mean = df_rolling_7day_mean.dropna()

### 3.5.3. Practice #8: Can you calculate the 7Q10 for Niagara River?
This is definitely a harder exercise! However, the philosophy behind the calculation is very similar to how we calculate the annual peak flow with 100-year recurrence interval.

7Q10 is the 7-day average annual low flow with 10-year recurrence interval.

1. Calculate the 7-day average flow (we have done in **3.5.1/3.5.2**)
2. Find the annual minimum 7-day average flow (use groupby, see Section **3.3.1**)
3. Calculate CDF, which equals to Annual Non-exceedance Probability (ANEP)!!
4. Find the flow corresponds to ANEP = 0.1 (which corresponds to 10-year recurrence interval)

In [ ]:
# Step 1: Identify annual 7-day low flow


In [ ]:
# Step 2: calculate cdf, which equals to anep


In [ ]:
# Step 3 (optional): visualize ANEP


In [ ]:
# Step 4: find the flow corresponds to ANEP = 0.1
